In [1]:
import ga4gh.vrs._internal.models
import importlib
importlib.reload(ga4gh.vrs._internal.models)
from ga4gh.vrs._internal.models import (
    Allele,
    SystemicVariation,
    flatten_type,
    class_refatt_map
)
from ga4gh.vrs import models


TODO

- Update pydantic.py helper functions to work for classes in models.py (if those functions are needed)
- Update Translator _from* functions to work for the simple Allele in test_vrs.py (also below in allele_dict)
- Update the ga4gh_serialize and ga4gh_digest functions to work with Pydantic classes and the ga4ghDigest field in VRS 2.0
- Update test in test_vrs.py to work for new serialize and digest functions, for update expected values for VRS 2.model
- location start/end validation in Pydantic class
- IRI validation in Pydantic class (curie, compacted identifier, or fully qualified IRI according to RFCs)


In [2]:

allele_dict = {
    'location': {
        'end': 55181320,
        'start': 55181319,
        'sequence': {
            'type': 'SequenceReference',
            'refgetAccession': 'SQ.F-LrLMe1SRpfUZHkQmvkVKFEGaoDeHul'
        },
        # 'sequence_id': 'ga4gh:SQ.F-LrLMe1SRpfUZHkQmvkVKFEGaoDeHul',
        'type': 'SequenceLocation'
    },
    'state': {
        'sequence': 'T',
        'type': 'LiteralSequenceExpression'
    },
    'type': 'Allele'
}

allele = Allele(**allele_dict)

In [3]:
# test pydantic_copy
from ga4gh.core._internal.pydantic import pydantic_copy
actual = pydantic_copy(allele).dict()
expected = allele.dict()
assert actual == expected # they have the same value
actual["foo"] = "bar"
assert actual != expected # they are not the same object

In [4]:
class_refatt_map

{'SequenceLocation': ['sequence'],
 'Allele': ['location'],
 'Haplotype': ['members'],
 'CopyNumberCount': ['subject'],
 'CopyNumberChange': ['subject'],
 'GenotypeMember': ['variation']}

In [9]:
import ga4gh.core._internal.identifiers
importlib.reload(ga4gh.core._internal.identifiers)
from ga4gh.core._internal.identifiers import (
    ga4gh_serialize,
    ga4gh_roll_up_refs
)

# ga4gh_roll_up_refs(allele.dict(), class_refatt_map)
ga4gh_serialize(allele)

b'{}'

In [ ]:
t = ga4gh.vrs._internal.models.Allele.__fields__["type"].type_
t

typing.Literal['Allele']

In [ ]:
str.split()

In [11]:
from ga4gh.vrs.extras.translator import Translator
from ga4gh.vrs.dataproxy import SeqRepoDataProxy
from biocommons.seqrepo import SeqRepo
data_proxy = SeqRepoDataProxy(SeqRepo("/Users/kferrite/dev/biocommons.seqrepo/seqrepo/2021-01-29"))
translator = Translator(data_proxy=data_proxy)
# translator._from_beacon("13 : 32936732 G > C")

In [12]:
data_proxy.get_metadata('GRCh38:13')

{'length': 114364328,
 'alphabet': 'ACGKNTY',
 'added': '2016-08-27T23:50:14Z',
 'aliases': ['GRCh38:13',
  'GRCh38:chr13',
  'GRCh38.p1:13',
  'GRCh38.p1:chr13',
  'GRCh38.p10:13',
  'GRCh38.p10:chr13',
  'GRCh38.p11:13',
  'GRCh38.p11:chr13',
  'GRCh38.p12:13',
  'GRCh38.p12:chr13',
  'GRCh38.p2:13',
  'GRCh38.p2:chr13',
  'GRCh38.p3:13',
  'GRCh38.p3:chr13',
  'GRCh38.p4:13',
  'GRCh38.p4:chr13',
  'GRCh38.p5:13',
  'GRCh38.p5:chr13',
  'GRCh38.p6:13',
  'GRCh38.p6:chr13',
  'GRCh38.p7:13',
  'GRCh38.p7:chr13',
  'GRCh38.p8:13',
  'GRCh38.p8:chr13',
  'GRCh38.p9:13',
  'GRCh38.p9:chr13',
  'MD5:a5437debe2ef9c9ef8f3ea2874ae1d82',
  'NCBI:NC_000013.11',
  'refseq:NC_000013.11',
  'SEGUID:2oDBty0yKV9wHo7gg+Bt+fPgi5o',
  'SHA1:da80c1b72d32295f701e8ee083e06df9f3e08b9a',
  'VMC:GS__0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'sha512t24u:_0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT']}

In [ ]:
#vars(ga4gh.vrs.models)

In [17]:
import re
re.match(r'[a-zA-Z0-9.]+:\S+', "ga4gh:asdf")

<re.Match object; span=(0, 10), match='ga4gh:asdf'>

In [41]:
from ga4gh.vrs import models
import pydantic

models.IRI(__root__="asdf").dict()



{'__root__': 'asdf'}